In [ ]:
! pip3 install python-nmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install -y nmap

Reading package lists... Done
Building dependency tree       
Reading state information... Done
nmap is already the newest version (7.80+dfsg1-2build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!which nmap

/usr/bin/nmap


In [ ]:
import os
os.environ['PATH'] = os.environ['PATH'] + ':/usr/bin/nmap'


In [ ]:
!apt install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
import shutil
dir_path = '/content/scipag_vulscan'
shutil.rmtree(dir_path)

FileNotFoundError: ignored

In [ ]:
# Cloning the repository and creating a symbolic link
!git clone https://github.com/scipag/vulscan scipag_vulscan
!ln -s `pwd`/scipag_vulscan /usr/share/nmap/scripts/vulscan


import os
import subprocess
import re

# Define the target IP address or domain name
target = "93.184.216.34"

# Define the Nmap command with vulscan script and run the command
nmap_cmd = f"nmap -Pn --script=vulscan/vulscan.nse {target}"
output = subprocess.check_output(nmap_cmd, shell=True)

# Extract the CVE numbers from the output using regex
cve_regex = re.compile(r'(CVE-\d{4}-\d{4,7})')
cve_matches = cve_regex.findall(output.decode())

# Print the CVE numbers
print("CVE numbers:")
for cve in cve_matches:
    print(cve)


Cloning into 'scipag_vulscan'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 282 (delta 6), reused 7 (delta 2), pack-reused 264
Receiving objects: 100% (282/282), 17.49 MiB | 27.90 MiB/s, done.
Resolving deltas: 100% (169/169), done.
CVE numbers:


In [ ]:
# Cloning the repository and creating a symbolic link
!git clone https://github.com/scipag/vulscan scipag_vulscan
!ln -s `pwd`/scipag_vulscan /usr/share/nmap/scripts/vulscan


import os
import subprocess

# The target IP address or domain name
target = "93.184.216.34"

# Nmap command with vulscan script and run the command
nmap_cmd = f"nmap -Pn -sV --script=vulscan/vulscan.nse {target}"
output = subprocess.check_output(nmap_cmd, shell=True)

print(output.decode())



Cloning into 'scipag_vulscan'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 282 (delta 6), reused 7 (delta 2), pack-reused 264
Receiving objects: 100% (282/282), 17.49 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (169/169), done.
Starting Nmap 7.80 ( https://nmap.org ) at 2023-02-21 05:21 UTC
Nmap scan report for 93.184.216.34
Host is up (0.011s latency).
Not shown: 996 filtered ports
PORT     STATE  SERVICE  VERSION
80/tcp   open   http     Edgecast CDN httpd (agb/A431)
| http-server-header: 
|   ECS (agb/5295)
|_  ECS (agb/A430)
| vulscan: VulDB - https://vuldb.com:
| No findings
| 
| MITRE CVE - https://cve.mitre.org:
| [CVE-2012-5159] phpMyAdmin 3.5.2.2, as distributed by the cdnetworks-kr-1 mirror during an unspecified time frame in 2012, contains an externally introduced modification (Trojan Horse) in server_sync.php, which allows remote attackers to execute arbitrary PHP 

In [ ]:
import os
import subprocess
from multiprocessing import Pool

# Clone the vulscan repository and create a symbolic link
subprocess.run("git clone https://github.com/scipag/vulscan scipag_vulscan", shell=True, check=True)
subprocess.run("sudo ln -s `pwd`/scipag_vulscan /usr/share/nmap/scripts/vulscan", shell=True, check=True)


# The input file containing a list of IP addresses
input_file = "/content/random_ips.txt"

# The file containing a list of CVEs to match against
cve_file = "/content/found_cves.txt"

# The number of parallel processes to run
num_processes = 4

# Function to run the Nmap command for a single IP and return the output
def run_nmap(ip):
    nmap_cmd = f"nmap -Pn -sV --script=vulscan/vulscan.nse {ip}"
    output = subprocess.check_output(nmap_cmd, shell=True)
    return output.decode()

# Read the list of IP addresses from the input file
with open(input_file, "r") as f:
    ip_list = f.read().splitlines()

# Use multiprocessing to run the Nmap commands in parallel
with Pool(processes=num_processes) as pool:
    nmap_results = pool.map(run_nmap, ip_list)

# Parse the output from each Nmap command and extract the CVE information
ip_cve_set = set()
for i, result in enumerate(nmap_results):
    ip = ip_list[i]
    cve_list = []
    for line in result.splitlines():
      if "CVE" in line:
        cve_parts = line.split(":")
        if len(cve_parts) > 1:
            cve = cve_parts[1].strip()
            cve_list.append(cve)
    if cve_list:
        ip_cve_set.add((ip, tuple(cve_list)))
    else:
        ip_cve_set.add((ip, ("No CVE information found",)))

# Read the list of CVEs to match against
with open(cve_file, "r") as f:
    cve_list = set(f.read().splitlines())

# Find the matches between the IP CVEs and the provided CVE list
matched_cve_set = set()
for ip, cve_tuple in ip_cve_set:
    for cve in cve_tuple:
        if cve in cve_list:
            matched_cve_set.add(cve)

# Print the IP addresses and their corresponding CVEs
for ip, cve_tuple in ip_cve_set:
    print(ip, ":", list(cve_tuple))

# Print the matched CVEs
print("Matched CVEs:", matched_cve_set)

# Remove the cloned repository and the symbolic link
subprocess.run("rm -rf scipag_vulscan", shell=True, check=True)
subprocess.run("rm /usr/share/nmap/scripts/vulscan", shell=True, check=True)


243.151.103.53 : ['No CVE information found']
93.184.216.34 : ['//cve.mitre.org', 'this can be used by arbitrary web sites to host exploit code that targets this control.', 'this can be leveraged for code execution by writing to a Startup folder.', '//cve.mitre.org', 'this can be used by arbitrary web sites to host exploit code that targets this control.', 'this can be leveraged for code execution by writing to a Startup folder.']
213.199.203.144 : ['No CVE information found']
164.77.244.121 : ['No CVE information found']
102.71.34.31 : ['No CVE information found']
231.12.20.191 : ['No CVE information found']
170.181.159.53 : ['No CVE information found']
90.150.199.218 : ['No CVE information found']
51.119.95.152 : ['No CVE information found']
220.230.98.176 : ['No CVE information found']
Matched CVEs: set()


CompletedProcess(args='rm /usr/share/nmap/scripts/vulscan', returncode=0)

In [ ]:
import os
import subprocess
from multiprocessing import Pool

# Clone the vulscan repository and create a symbolic link
subprocess.run("git clone https://github.com/scipag/vulscan scipag_vulscan", shell=True, check=True)
subprocess.run("sudo ln -s `pwd`/scipag_vulscan /usr/share/nmap/scripts/vulscan", shell=True, check=True)


# The input file containing a list of IP addresses
input_file = "/content/random_ips.txt"

# The file containing a list of CVEs to match against
cve_file = "/content/found_cves.txt"

# The number of parallel processes to run
num_processes = 4

# Function to run the Nmap command for a single IP and return the output
def run_nmap(ip):
    nmap_cmd = f"nmap -Pn -sV --script=vulscan/vulscan.nse {ip}"
    output = subprocess.check_output(nmap_cmd, shell=True)
    return output.decode()

# Read the list of IP addresses from the input file
with open(input_file, "r") as f:
    ip_list = f.read().splitlines()

# Use multiprocessing to run the Nmap commands in parallel
with Pool(processes=num_processes) as pool:
    nmap_results = pool.map(run_nmap, ip_list)

# Parse the output from each Nmap command and extract the CVE information
ip_cve_set = set()
for i, result in enumerate(nmap_results):
    ip = ip_list[i]
    cve_list = []
    for line in result.splitlines():
        if "CVE" in line:
            cve_parts = line.split(":")
            if len(cve_parts) > 1:
                cve = cve_parts[1].strip()
                cve_desc = line.split("CVE-")[0].strip()
                cve_list.append((cve, cve_desc))
    if cve_list:
        ip_cve_set.add((ip, tuple(cve_list)))
    else:
        ip_cve_set.add((ip, (("No CVE information found", ""),)))

# Read the list of CVEs to match against
with open(cve_file, "r") as f:
    cve_list = set(f.read().splitlines())

# Find the matches between the IP CVEs and the provided CVE list
matched_cve_set = set()
for ip, cve_tuple in ip_cve_set:
    for cve, desc in cve_tuple:
        if cve in cve_list:
            matched_cve_set.add((cve, desc))

# Print the IP addresses and their corresponding CVEs
for ip, cve_tuple in ip_cve_set:
    print(ip, ":")
    for cve, desc in cve_tuple:
        print(f"\tCVE: {cve}\n\tDescription: {desc}")

# Print the matched CVEs
print("Matched CVEs:")
for cve, desc in matched_cve_set:
    print(f"\tCVE: {cve}\n\tDescription: {desc}")

# Remove the cloned repository and the symbolic link
subprocess.run("rm -rf scipag_vulscan", shell=True, check=True)
subprocess.run("rm /usr/share/nmap/scripts/vulscan", shell=True, check=True)


220.230.98.176 :
	CVE: No CVE information found
	Description: 
90.150.199.218 :
	CVE: No CVE information found
	Description: 
243.151.103.53 :
	CVE: No CVE information found
	Description: 
213.199.203.144 :
	CVE: No CVE information found
	Description: 
170.181.159.53 :
	CVE: No CVE information found
	Description: 
231.12.20.191 :
	CVE: No CVE information found
	Description: 
164.77.244.121 :
	CVE: No CVE information found
	Description: 
93.184.216.34 :
	CVE: //cve.mitre.org
	Description: | MITRE CVE - https://cve.mitre.org:
	CVE: this can be used by arbitrary web sites to host exploit code that targets this control.
	Description: | [
	CVE: this can be leveraged for code execution by writing to a Startup folder.
	Description: | [
	CVE: //cve.mitre.org
	Description: | MITRE CVE - https://cve.mitre.org:
	CVE: this can be used by arbitrary web sites to host exploit code that targets this control.
	Description: | [
	CVE: this can be leveraged for code execution by writing to a Startup folde

CompletedProcess(args='rm /usr/share/nmap/scripts/vulscan', returncode=0)

In [ ]:
import os
import subprocess
from multiprocessing import Pool

# Clone the vulscan repository and create a symbolic link
subprocess.run("git clone https://github.com/scipag/vulscan scipag_vulscan", shell=True, check=True)
subprocess.run("sudo ln -s `pwd`/scipag_vulscan /usr/share/nmap/scripts/vulscan", shell=True, check=True)


# The input file containing a list of IP addresses
input_file = "/content/random_ips.txt"

# The file containing a list of CVEs to match against
cve_file = "/content/found_cves.txt"

# The number of parallel processes to run
num_processes = 6

# Function to run the Nmap command for a single IP and return the output
def run_nmap(ip):
    nmap_cmd = f"nmap -Pn -sV --script=vulscan/vulscan.nse {ip}"
    output = subprocess.check_output(nmap_cmd, shell=True)
    return output.decode()

# Read the list of IP addresses from the input file
with open(input_file, "r") as f:
    ip_list = f.read().splitlines()

# Use multiprocessing to run the Nmap commands in parallel
with Pool(processes=num_processes) as pool:
    nmap_results = pool.map(run_nmap, ip_list)

# Parse the output from each Nmap command and extract the CVE information
ip_cve_set = set()
for i, result in enumerate(nmap_results):
    ip = ip_list[i]
    cve_list = []
    for line in result.splitlines():
        if "CVE" in line:
            cve_parts = line.split(":")
            if len(cve_parts) > 1:
                cve = cve_parts[1].strip()
                cve_desc = line.split("CVE-")[0].strip()
                cve_list.append((cve, cve_desc))
    if cve_list:
        ip_cve_set.add((ip, tuple(cve_list)))
    else:
        ip_cve_set.add((ip, (("No CVE information found", ""),)))

# Read the list of CVEs to match against
with open(cve_file, "r") as f:
    cve_list = set(f.read().splitlines())

# Find the matches between the IP CVEs and the provided CVE list
matched_cve_set = set()
for ip, cve_tuple in ip_cve_set:
    for cve, desc in cve_tuple:
        if cve in cve_list:
            matched_cve_set.add((cve, desc))

# Print the IP addresses and their corresponding CVEs
for ip, cve_tuple in ip_cve_set:
    print(ip, ":")
    for cve, desc in cve_tuple:
        print(f"\tCVE: {cve}\n\tDescription: {desc}")

# Print the matched CVEs
print("Matched CVEs:")
for cve, desc in matched_cve_set:
    print(f"\tCVE: {cve}\n\tDescription: {desc}")

# Remove the cloned repository and the symbolic link
subprocess.run("rm -rf scipag_vulscan", shell=True, check=True)
subprocess.run("rm /usr/share/nmap/scripts/vulscan", shell=True, check=True)


220.230.98.176 :
	CVE: No CVE information found
	Description: 
90.150.199.218 :
	CVE: No CVE information found
	Description: 
243.151.103.53 :
	CVE: No CVE information found
	Description: 
213.199.203.144 :
	CVE: No CVE information found
	Description: 
170.181.159.53 :
	CVE: No CVE information found
	Description: 
231.12.20.191 :
	CVE: No CVE information found
	Description: 
164.77.244.121 :
	CVE: No CVE information found
	Description: 
93.184.216.34 :
	CVE: //cve.mitre.org
	Description: | MITRE CVE - https://cve.mitre.org:
	CVE: this can be used by arbitrary web sites to host exploit code that targets this control.
	Description: | [
	CVE: this can be leveraged for code execution by writing to a Startup folder.
	Description: | [
	CVE: //cve.mitre.org
	Description: | MITRE CVE - https://cve.mitre.org:
	CVE: this can be used by arbitrary web sites to host exploit code that targets this control.
	Description: | [
	CVE: this can be leveraged for code execution by writing to a Startup folde

CompletedProcess(args='rm /usr/share/nmap/scripts/vulscan', returncode=0)